In [1]:
########IMPORTANTE########
#O ARQUIVO DE DADOS POSSUI 144Mbytes, PORTANTO NÃO PODE SER ENVIADO PELO GITHUB. O LINK DELE ESTÁ DISPONIBILIZADO ABAIXO:
#https://www.kaggle.com/dalpozz/creditcardfraud

In [2]:
%matplotlib inline
import pandas as pd
import math
import matplotlib.pyplot as plt
import numpy as np
import os
from numpy import zeros_like
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix,auc,roc_auc_score
from sklearn.metrics import recall_score, precision_score, accuracy_score, f1_score

In [3]:
df = pd.read_csv('creditcard.csv',sep=",")

In [4]:
len(df)

284807

In [5]:
# separando dataset em base de treinamento e em base de teste


clas = df['Class'].values #fraude ou não
feat = df.drop(['Class'],axis=1).values #features
feat_train, feat_test, clas_train, clas_test = train_test_split(feat, clas, test_size=0.3, stratify=clas)

print("Tamanho da base treinamento: ", len(clas_train))
print("Fraudes na base treinamento: ", sum(clas_train))
print("Tamanho da base teste: ", len(clas_test))
print("Fraudes na base teste: ", sum(clas_test))

Tamanho da base treinamento:  199364
Fraudes na base treinamento:  344
Tamanho da base teste:  85443
Fraudes na base teste:  148


In [6]:
# classificando base teste
clf = GaussianNB()
clf.fit(feat_train,clas_train)

prob_fraude = clf.predict_proba(feat_test)


In [7]:
# Tomada de decisão

classificador1 = pd.DataFrame(clas_test)


for index, row in classificador1.iterrows():
    i = classificador1.index.get_loc(index)
    if prob_fraude[i][0]>prob_fraude[0][1]:
        classificador1.loc[index, 'Classificação'] = '0'


    if prob_fraude[i][0]<prob_fraude[0][1]:
        classificador1.loc[index, 'Classificação'] = '1'

In [8]:
Comparativo = pd.crosstab(classificador1[0],classificador1['Classificação'])
Comparativo

Classificação,0,1
0,,
0,85105,190
1,84,64
